# Back test of Binance trader container

## Resumo do algoritmo:

### 1 - Entrada

    1 - Lê as tabelas de candles (para os pares monitorados) como dataframes, criando um dicionário para os pares:
    
    {'par':'dataframe'}
    
### 2 - Ciclos

    0 - Adquire os parâmetros, como o número mínimo de amostras para o correto cálculo de todos os indicadores
        daquela amostra, naquele instante.
        
        [Principais parâmetros neste caso]
        
            - Candle_interval
            - Número de amostras para média móvel curta
            - Número de amostras para média móvel Longa
            - Critérios de stop, etc
    
    1 - Para cada par no dicionário:
    
        (completa os dataframes, confome tenham chegados novos dados)
        
        (AVALIA A PARTIR DE QUAL AMOSTRA TRABALHAR. Para tanto deve buscar a <tabela_monitoramento_bt> no banco de
        dados)
        
        1.0 - Calcula i_min e i_max a partir do Candle_interval, etc
        
        1.1 - Para i no range de (i_min, i_max)
        
            (AVALIA CRITÉRIOS DE STOP)
        
            1.1.1 - Faz o parseamento de dados para a escala do candle_interval trabalhado, criando novo dataframe
            
            1.1.2 - Entrga este dataframe ao módulo <definicao_posicao>
            
                *=======================================================================================*
                |                                                                                       |
                | <definicao_posicao>                                                                   |
                | ===================                                                                   |
                |                                                                                       |
                |    I - Para todos os indicadores utilizados por este módulo:                          |
                |                                                                                       |
                |       I.1 - Entrega o dataframe ao módulo <indicador>, que retorna uma posição        |
                |                                                                                       |
                |   II - Avalia as posições dadas pelos indicadores e toma uma decição sobre a posição  |
                |                                                                                       |
                | Uma posição, para este par, é retornada                                               |
                *---------------------------------------------------------------------------------------*

       
        1.2 - Cria o dicionário {'par':'posição'} dos pares e suas posições, gravando as entradas pertinentes
            na tabela <tabela_monitoramento_bt>.
        
        1.3 - Entrega este dataframe ao módulo <trader_action> que deve avaliar o disparamento ou não de uma ordem
        
            1.3.1 - <trader_action>
            
                I - (AVALIA CRITÉRIOS DE STOP)
            
            
            
            
            
            
            
            
            
            
            

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import os

In [3]:
#Credenciais do banco

POSTGRES_DB='pg_dev'
POSTGRES_PASSWORD='06Fj@%r7KTXm5+eWn2'
POSTGRES_USER='openbot'
DB_HOST='localhost'
DB_PORT='5432'

db_url = 'postgresql://' + POSTGRES_USER + ':' + POSTGRES_PASSWORD + '@' + DB_HOST + ':' + DB_PORT + '/' + POSTGRES_DB

#print(db_url)

engine = create_engine(db_url,echo=False)

In [ ]:
#Ler do banco

table_name = 'binance_klines_btcusdt_1m'

sql_query = 'SELECT * FROM ' + table_name

btcusdt_1m_klines = pd.read_sql(sql_query, engine)

In [ ]:
#Salvar no banco

In [4]:
#Modelo de como criar dataframes dinâmicos, um para cada tabela, e colecioná-los em um dicionário

#Modelo
d={}
for x in range(1,10):
        d["string{0}".format(x)]="Hello"

#Teste
table_name = 'binance_pairs'

sql_query = 'SELECT * FROM ' + table_name

colecao={}
for i in range(1,10):
    
        colecao["pares{0}".format(i)] = pd.read_sql(sql_query, engine)

In [8]:
colecao['pares3']

,name,get_data,status,last_change_by_pid
0,LTCUSDT,ON,full,8282
1,LINKUSDT,ON,full,8285
2,BTCUSDT,ON,full,8284
3,ETHUSDT,ON,full,8283
4,NEOUSDT,ON,full,8281
5,XRPUSDT,ON,full,8286


In [ ]:
#Candle_interval = 2h
#Preço = (o + h + l + c)/4
#Media_longa = 100 amostras
#Media_curta = 3 amostras
#Ordem a mercado (entra e sai pelo preço do minuto seguinte)

'''
A cada ciclo (iniciando de uma posição sucifientemente longa para poder calcular o valor do indicador) avalia 
em que posição deve estar o ativo, colocando um sinal (na tabela - dicionário - de ativos em operação) em caso 
de a posição precisar ser modificada
'''